In [ ]:
# opengrid imports
from opengrid.library import misc, houseprint, caching
from opengrid.library.analysis import DailyAgg
from opengrid import config
from opengrid.library.slack import Slack
from opengrid.library import alerts
c=config.Config()

# other imports
import pandas as pd
import json
import charts
import numpy as np
import os
import datetime as dt
import pytz
BXL = pytz.timezone('Europe/Brussels')


# configuration for the plots
DEV = c.get('env', 'type') == 'dev' # DEV is True if we are in development environment, False if on the droplet
print("Environment configured for development: {}".format(DEV))
if not DEV:
    # production environment: don't try to display plots
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.dates import MinuteLocator, HourLocator, DateFormatter, AutoDateLocator, num2date

if DEV:
    if c.get('env', 'plots') == 'inline':
        %matplotlib inline
    else:
        %matplotlib qt
else:
    pass # don't try to render plots
plt.rcParams['figure.figsize'] = 12,8

In [ ]:
hp = houseprint.Houseprint()
sensors = hp.get_sensors(sensortype='electricity') # sensor objects

# Remove some sensors
exclude = [
            '565de0a7dc64d8370aa321491217b85f' # 3E
          ]
solar = [x.key for x in hp.search_sensors(type='electricity', system='solar')]
exclude += solar

for s in sensors:
    if s.key in exclude:
        sensors.remove(s)

hp.init_tmpo()

In [ ]:
hp.sync_tmpos()

In [ ]:

sensors = hp.get_sensors() # sensor objects
t = []
for s in sensors:
    key = s.key
    lt = s.last_timestamp()
    if lt is None:
        continue
    nr_of_days = pd.Timestamp('now', tz='UTC') - lt
    t.append((key,nr_of_days))


In [ ]:
df = pd.DataFrame(t).rename(columns={0: "sensor_id", 1: "timedelta"}).set_index('sensor_id')

In [ ]:
df['seconds'] = df.timedelta.map(lambda x: x.total_seconds())
df['days'] = df.timedelta.map(lambda x: x.days)

In [ ]:
df

# Setup NoDataBot slack bot

In [ ]:
slack_url = c.get('Slack', 'webhook')
username = 'NoDataBot'
channel = "junk" # we don't want to clutter up everything
emoji = ':warning:'
slack = Slack(url=slack_url, username=username, channel=channel, emoji=emoji)

# Create the alerts and send

In [ ]:
alerts.create_alerts(df, hp, 'no_sensor_data', slack, column='seconds')

In [ ]:
import pdb; pdb.pm()